In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
from spacy import displacy
from pathlib import Path

import seaborn as sns
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.pipeline import Pipeline
from string import punctuation

import joblib
import eli5
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, classification_report,roc_auc_score
from sklearn import metrics
import random
from spacy import displacy

seed = 123
random.seed(seed)

import nltk
import spacy
import string

#!pip install contractions
import contractions

from nltk.corpus import stopwords                   # Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer
from nltk.stem import WordNetLemmatizer             # Lemmatization

from nltk.corpus import wordnet                     # Wordnet for synonyms & antonyms
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize             # Word Tokenizer
from nltk import FreqDist

from collections import Counter

#from sklearn.feature_extraction.text import CountVectorizer          #For Bag of words
#from sklearn.feature_extraction.text import TfidfVectorizer          #For TF-IDF
#from nltk import ngrams                                              # ngrams
#from gensim.models import Word2Vec                                   #For Word2Vec

Creating our own custom entity taging specificially for drugs

In [2]:
#NEED TO INCLUDE THE DRUG NAMES FROM OUR SUBSET
drug_keywords = ["drug", "pill", "tablet", "medicine", "patch", "injection", "prescription", "treatment",
        "medication", "capsule", "serum", "lotion", "ointment", "tincture", "tonic", "vaccination",
        "vaccine", "birth control", "contraceptive", "Lybrel","Lybrel","Nexplanon","nexplanon", "decrease", "lessen", "reduce", "drop",
        "diminish", "dwindle", "shrink", "subside", "cut", "reduction", "drop", "decline", "increase",
        "improve", "grow", "elevate", "decreased", "lessened", "reduced", "dropped",
        "diminished", "dwindled", "shrinked", "subsided", "cut", "declined", "increased",
        "improved", "grew", "elevated", "decreasing", "lessening", "reducing", "dropping",
        "diminishing", "dwindling", "shrinking", "subsiding", "cutting", "declining", "increasing",
        "improving", "growing", "elevating"]

In [3]:
#make keywords into dictionary
drug_terms = {}
for ele in drug_keywords:
    drug_terms[ele] = []
drug_terms

{'drug': [],
 'pill': [],
 'tablet': [],
 'medicine': [],
 'patch': [],
 'injection': [],
 'prescription': [],
 'treatment': [],
 'medication': [],
 'capsule': [],
 'serum': [],
 'lotion': [],
 'ointment': [],
 'tincture': [],
 'tonic': [],
 'vaccination': [],
 'vaccine': [],
 'birth control': [],
 'contraceptive': [],
 'Lybrel': [],
 'Nexplanon': [],
 'nexplanon': [],
 'decrease': [],
 'lessen': [],
 'reduce': [],
 'drop': [],
 'diminish': [],
 'dwindle': [],
 'shrink': [],
 'subside': [],
 'cut': [],
 'reduction': [],
 'decline': [],
 'increase': [],
 'improve': [],
 'grow': [],
 'elevate': [],
 'decreased': [],
 'lessened': [],
 'reduced': [],
 'dropped': [],
 'diminished': [],
 'dwindled': [],
 'shrinked': [],
 'subsided': [],
 'declined': [],
 'increased': [],
 'improved': [],
 'grew': [],
 'elevated': [],
 'decreasing': [],
 'lessening': [],
 'reducing': [],
 'dropping': [],
 'diminishing': [],
 'dwindling': [],
 'shrinking': [],
 'subsiding': [],
 'cutting': [],
 'declining': []

# Initiate Spacy

In [4]:
# Review ID #13800 as an example
example = "This is my first time using any form of birth control. I'm glad I went with the patch, I have been on it for 8 months? At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period! I never had cramps before using birth control. Other than that in happy with the patch"
example_2 = "This is my first time using any form of birth control. I am glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch."


In [5]:
nlp = spacy.load("en_core_web_sm")
#doc=nlp(example)
doc = nlp(example_2)
displacy.render(doc, style="ent")

In [6]:
#nlp = spacy.load("en_core_web_sm", parse = True, tag = True, entity = True)
displacy.render(doc, style = 'dep')

In [7]:
# Save the example as a image file
img = displacy.render(doc, style = 'dep', jupyter = False)
output_path = Path('./images/dependency_plot.svg')
output_path.open('w', encoding = 'utf-8').write(str(img))

74068

# Pre-Process reviews

In [8]:
# Initiate Spacy# Removing stopwords
#nltk.download('stopwords') ## uncomment if stopwords hasn't been installed yet

STOPWORDS = set(stopwords.words('english')) ## might need to customize stopwords for Edwards ##

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return ' '.join([word for word in str(text).split() if word not in STOPWORDS])

#clean_df['clean_review'] = clean_df['clean_review'].apply(lambda text: remove_stopwords(text))
#clean_df.head(10)
# Stemming
stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

#clean_df['clean_review'] = clean_df['clean_review'].apply(lambda text: stem_words(text))
#clean_df.head(10)
# Lemmatization
#nltk.download('wordnet') ## uncomment if wordnet hasn't been installed yet
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

#clean_df['clean_review'] = clean_df['clean_review'].apply(lambda text: lemmatize_words(text))
#clean_df.head(10)


In [9]:
%%time
df = pd.read_csv('drugsComTrain_raw.csv')
# Create a copy of the original df
clean_df = df

# Sub '&#039;' with ' since it's a code for contraction 
clean_df['clean_review'] = clean_df['review'].apply(lambda x: re.sub(r'&#039;', "'", x))

# Sub '&amp;' with 'and' since it's a code for and 
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'&amp;', "and", x))

# Sub '&quo;' with '""' since it's a code for quotations
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'&quot;', '"', x))

# Remove quotations before and after a string
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'^"|"$', "", x))

# Remove extra whitespaces inside the review
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r"\s+", " ", x))

# Remove any trailing whitespaces before and after string
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: x.strip())

#lemmatize word
clean_df['lemm'] = clean_df['clean_review'].apply(lambda x: lemmatize_words(x))

# Expanding Contractions
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: contractions.fix(x))

# Remove any punctations
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

# Remove digits and words containing digits
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'\w*\d\w*','', x))

# Lower case
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: x.lower())

# Remove extra whitespaces inside the review
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r"\s+", " ", x))

# Remove any trailing whitespaces before and after string
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: x.strip())

# Remove any trailing whitespaces before and after string
clean_df['rmv_stop'] = clean_df['clean_review'].apply(lambda x: remove_stopwords(x))

clean_df['clean_review'].head(10)

# Subset the drug dataset by the Top 5 Conditions
top5 = ['Depression', 'Birth Control', 'Anxiety', 'Pain', 'Acne']
top5_conditions = clean_df[clean_df['condition'].isin(top5)].reset_index()

# Double-Check
#top5_conditions['condition'].unique()
#top5_conditions.info()
#top5_conditions.shape ## 55,494 rows & 9 columns

display(top5_conditions.head(5))

# new_df = top5_conditions.loc[top5_conditions['drugName']=="Nexplanon"]
# new_df.shape

,index,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review,lemm,rmv_stop
0,2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,i used to take another oral contraceptive whic...,"I used to take another oral contraceptive, whi...",used take another oral contraceptive pill cycl...
1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,this is my first time using any form of birth ...,This is my first time using any form of birth ...,first time using form birth control glad went ...
2,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,i had been on the pill for many years when my ...,I had been on the pill for many years. When my...,pill many years doctor changed rx chateal effe...
3,11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,9-Mar-17,54,i have taken anti depressants for years with s...,"I have taken anti-depressants for years, with ...",taken anti depressants years improvement mostl...
4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,started nexplanon months ago because i have a ...,Started Nexplanon 2 month ago because I have a...,started nexplanon months ago minimal amount co...


Wall time: 2min 21s


In [10]:
top5_conditions.condition.value_counts()

Birth Control    28788
Depression        9069
Pain              6145
Anxiety           5904
Acne              5588
Name: condition, dtype: int64

# POS Tagging

In [11]:
# Pre-Process reviews# Using the 1st review of top5_conditions as an example

words = word_tokenize(top5_conditions.clean_review[0])
tagged = nltk.pos_tag(words)


for x in tagged:
    print(x)
    
    # Find a set of synonyms based on the 1st review of the top5_condition

#words = []
STOPWORDS = set(stopwords.words('english')) # REMOVE STOPWORDS
synonyms = [] # Create a list of synonyms

for word in top5_conditions.clean_review[0].split():
    if word not in STOPWORDS:
        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())

print(set(synonyms))
#display(len(set(synonyms))) ## 741 synonym words

('i', 'NN')
('used', 'VBN')
('to', 'TO')
('take', 'VB')
('another', 'DT')
('oral', 'JJ')
('contraceptive', 'NN')
('which', 'WDT')
('had', 'VBD')
('pill', 'NN')
('cycle', 'NN')
('and', 'CC')
('was', 'VBD')
('very', 'RB')
('happy', 'JJ')
('very', 'RB')
('light', 'JJ')
('periods', 'NNS')
('max', 'VBP')
('days', 'NNS')
('no', 'DT')
('other', 'JJ')
('side', 'NN')
('effects', 'NNS')
('but', 'CC')
('it', 'PRP')
('contained', 'VBD')
('hormone', 'NN')
('gestodene', 'NN')
('which', 'WDT')
('is', 'VBZ')
('not', 'RB')
('available', 'JJ')
('in', 'IN')
('us', 'PRP')
('so', 'RB')
('i', 'RB')
('switched', 'VBD')
('to', 'TO')
('lybrel', 'VB')
('because', 'IN')
('the', 'DT')
('ingredients', 'NNS')
('are', 'VBP')
('similar', 'JJ')
('when', 'WRB')
('my', 'PRP$')
('other', 'JJ')
('pills', 'NNS')
('ended', 'VBD')
('i', 'RB')
('started', 'VBN')
('lybrel', 'NN')
('immediately', 'RB')
('on', 'IN')
('my', 'PRP$')
('first', 'JJ')
('day', 'NN')
('of', 'IN')
('period', 'NN')
('as', 'IN')
('the', 'DT')
('instructio

# Subset just one Drug:
Nexplanon and determine Word sentiment and weight (ELI)

In [12]:
top_n=10
top5_conditions.drugName.value_counts()[0:top_n].reset_index()

,index,drugName
0,Etonogestrel,3314
1,Ethinyl estradiol / norethindrone,2466
2,Nexplanon,2149
3,Levonorgestrel,2129
4,Ethinyl estradiol / norgestimate,1795
5,Ethinyl estradiol / levonorgestrel,1600
6,Implanon,1095
7,Mirena,965
8,Escitalopram,864
9,Skyla,822


In [13]:
top5_conditions["pos_sentiment"] = top5_conditions.rating.apply(lambda x: 1 if x >=7 else 0,1)
top5_conditions

,index,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review,lemm,rmv_stop,pos_sentiment
0,2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,i used to take another oral contraceptive whic...,"I used to take another oral contraceptive, whi...",used take another oral contraceptive pill cycl...,0
1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,this is my first time using any form of birth ...,This is my first time using any form of birth ...,first time using form birth control glad went ...,1
2,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,i had been on the pill for many years when my ...,I had been on the pill for many years. When my...,pill many years doctor changed rx chateal effe...,1
3,11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,9-Mar-17,54,i have taken anti depressants for years with s...,"I have taken anti-depressants for years, with ...",taken anti depressants years improvement mostl...,1
4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,started nexplanon months ago because i have a ...,Started Nexplanon 2 month ago because I have a...,started nexplanon months ago minimal amount co...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
55489,161273,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,i have had the nexplanon since december i got ...,"I have had the Nexplanon since Dec. 27, 2016 I...",nexplanon since december got first period end ...,0
55490,161276,176146,Lorazepam,Anxiety,"""About 4 years ago I started having early-morn...",8,21-Nov-17,0,about years ago i started having early morning...,About 4 year ago I started having early-mornin...,years ago started early morning awakening inso...,1
55491,161286,93069,Vortioxetine,Depression,"""This is the third med I&#039;ve tried for anx...",2,17-Jul-16,33,this is the third med i have tried for anxiety...,This is the third med I've tried for anxiety a...,third med tried anxiety mild depression week h...,0
55492,161287,132177,Ativan,Anxiety,"""I was super against taking medication. I&#039...",9,16-Aug-16,61,i was super against taking medication i have s...,I wa super against taking medication. I've sta...,super taking medication started dealing anxiet...,1


In [14]:
%%time

#should we use the subset dataframe (just the top 5 reviews) or entire dataframe?
#Currently only looking at 'Nexplanon' in the 'top5_conditions' dataframe
nexplanon_df = top5_conditions.loc[top5_conditions['drugName']=="Nexplanon"]

X,y = nexplanon_df.rmv_stop, nexplanon_df.pos_sentiment
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = seed)

Wall time: 70 ms


In [15]:
def eval_pipe(y_test, y_predict):
    print('\nOverall Accuracy Score: {}',metrics.accuracy_score(y_test, y_predict))
    print('\nConfusion Matrix: \n{}'.format(confusion_matrix(y_test, y_predict)))
    print('\nClassification Report: \n{}'.format(classification_report(y_test, y_predict)))
    print('\nROC-AUC Score: {}'.format(roc_auc_score(y_test,y_predict)))
    print("-----------------------------------------------------------------")

In [16]:
%%time
tfidf_bnb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('bnb', BernoulliNB(alpha = 1.0))
    ])
vect_bnb_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1,3))),
    ('bnb', BernoulliNB(alpha = 1.0))
    ])

tfidf_lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range = (1,3))),
    ('lr', LogisticRegression(C=0.1,max_iter=10000))
    ])
vect_lr_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1,3))),
    ('lr', LogisticRegression(C=0.1,max_iter=10000))
    ])

tfidf_bnb_pipe.fit(X_train, y_train)
tfidf_bnb_predict = tfidf_bnb_pipe.predict(X_test)

vect_bnb_pipe.fit(X_train, y_train)
vect_bnb_predict = vect_bnb_pipe.predict(X_test)

tfidf_lr_pipe.fit(X_train, y_train)
tfidf_lr_predict = tfidf_lr_pipe.predict(X_test)

vect_lr_pipe.fit(X_train, y_train)
vect_lr_predict = vect_lr_pipe.predict(X_test)

print("BernoulliNB,TFIDF Model Evaluation:")
eval_pipe(y_test, tfidf_bnb_predict)

print("BernoulliNB,Count Vecorizer Model Evaluation:")
eval_pipe(y_test, vect_bnb_predict)


print("Logistic RegressionTFIDF Model Evaluation:")
eval_pipe(y_test, tfidf_lr_predict)

print("Logistic Regression,Count Vecorizer Model Evaluation:")
eval_pipe(y_test, vect_lr_predict)

BernoulliNB,TFIDF Model Evaluation:

Overall Accuracy Score: {} 0.5465116279069767

Confusion Matrix: 
[[212   0]
 [195  23]]

Classification Report: 
              precision    recall  f1-score   support

           0       0.52      1.00      0.68       212
           1       1.00      0.11      0.19       218

    accuracy                           0.55       430
   macro avg       0.76      0.55      0.44       430
weighted avg       0.76      0.55      0.43       430


ROC-AUC Score: 0.5527522935779816
-----------------------------------------------------------------
BernoulliNB,Count Vecorizer Model Evaluation:

Overall Accuracy Score: {} 0.5465116279069767

Confusion Matrix: 
[[212   0]
 [195  23]]

Classification Report: 
              precision    recall  f1-score   support

           0       0.52      1.00      0.68       212
           1       1.00      0.11      0.19       218

    accuracy                           0.55       430
   macro avg       0.76      0.55      0.4

In [17]:
#display for visual purposes but tfidf is less accurate than count vectorizer
eli5_nex_tfidf = eli5.show_weights(tfidf_lr_pipe, top=50, target_names=y_test)
eli5_nex_tfidf

Weight?,Feature
+0.454,love
+0.303,far
+0.297,period
+0.194,nothing
+0.180,light
+0.175,pill
+0.168,insertion
+0.161,little
+0.146,period since
+0.143,reviews


In [18]:
eli5_nex_vect = eli5.show_weights(vect_lr_pipe, top=50, target_names=y_test)
eli5_nex_vect

Weight?,Feature
+0.906,love
+0.595,far
+0.410,nothing
+0.383,best
+0.350,light
+0.334,little
+0.326,reviews
+0.320,period
+0.292,remember
+0.276,overall


In [19]:
temp_df = pd.DataFrame(X_test)
temp_df

,rmv_stop
14417,nexplanon since july getting removed tomorrow ...
33363,got nexplanon year half ago periods stopped im...
37314,originally nexplanon implant fitted years ago ...
27920,terms birth control method product works well ...
25595,got birth control march shortly got heavy blee...
...,...
6054,nexolanon since end february period first two ...
43457,got nexplanon raging teen hormones want get pr...
50281,thing arm almost week feel like basket case cr...
40256,thing two years gained pounds biggest ever tho...


In [20]:
%%time
#if we want to run this by drug, we may exceed the current 3M limit. We will need to increase that
#need to call list(flatten(up_and_down_stream(review))) to get final list

from nltk.tokenize.treebank import TreebankWordDetokenizer

def up_and_down_stream_v7(review):
    nlp.max_length = 2000000
    doc = nlp(review)
    
    drug_terms = {}
    for ele in drug_keywords:
        drug_terms[ele] = []

    for token in doc:
        parent = token.head

        children = [child for child in token.children]
        siblings = [child for child in parent.children]
        
        if token.text in drug_terms.keys():
            string = TreebankWordDetokenizer().detokenize([token.text])
            drug_terms[string].append(parent)

            drug_terms[string].append([c for c in children])
            drug_terms[string].append([s for s in siblings])

    return drug_terms

def flatten(l):
    for i in l:
        if isinstance(i,list):
            yield from flatten(i)
        else:
            yield i

Wall time: 0 ns


# Create DataFrame for: Drug, Reviews, Related Words


In [21]:
%%time
#product, review, keykey terms

def paper_trail(condition,top_n=10):
    
    customer_feedback = []
    quality_control = []
    
    temp_df = top5_conditions.loc[top5_conditions['condition'] == condition]
    temp_drugs = temp_df.drugName.value_counts()[0:top_n].reset_index()
    top_product = list(temp_drugs["index"])

    for product in top_product:
        product_subset = temp_df.loc[temp_df['drugName']==product]
        product_reviews = product_subset.rmv_stop.str.cat(sep=' ')
        customer_feedback.append(product_reviews)
        
        terms = up_and_down_stream_v7(product_reviews)
        product_terms = {k: list(flatten(v)) for k, v in terms.items() if v} 
        quality_control.append(product_terms)
        
        
    return top_product,customer_feedback,quality_control

Wall time: 0 ns


In [22]:
%%time
def condition_df(condition = "missing",top_drugs = 10):
    top_product,customer_feedback,quality_control = paper_trail(condition, top_n=top_drugs)

    related_words =  []

    for idx in range(len(quality_control)):
        dict_temp = quality_control[idx]

        for key, ele in dict_temp.items():
            for obj in ele:
                if key == obj.text:
                    dict_temp[key].remove(obj)

        related_words.append(dict_temp)

    # merge lists into dictionaries
    merchandise_review = pd.DataFrame(
        {'Primo_Product': top_product,
         'Customer_Feedback': customer_feedback,
         'Quality_Control': related_words
        })

    return merchandise_review


Wall time: 0 ns


In [23]:
def display_df(dataframe):
    for idx,drug in enumerate(dataframe.Primo_Product):

        temp_drug_dict = dataframe.loc[idx,"Quality_Control"]
        print("{}:\n----------------------------------------------------------------------------------------\n ".format(drug))

        for k in temp_drug_dict.keys():
            print("Key:{}, Values:{}\n".format(k,temp_drug_dict[k]))

In [24]:
top5_conditions.condition.value_counts()

Birth Control    28788
Depression        9069
Pain              6145
Anxiety           5904
Acne              5588
Name: condition, dtype: int64

In [25]:
%%time
#run this cell you output all related words dataframe into some object
#access this object as you would any dataframe
bc_df = condition_df(condition = "Birth Control",top_drugs = 5)
depression_df = condition_df(condition = "Depression",top_drugs = 5)
pain_df = condition_df(condition = "Pain",top_drugs = 5)
anxiety_df = condition_df(condition = "Anxiety",top_drugs = 5)
acne_df = condition_df(condition = "Acne",top_drugs = 5)

Wall time: 2min 43s


In [26]:
bc_df

,Primo_Product,Customer_Feedback,Quality_Control
0,Etonogestrel,nexplanon job worry free sex thing periods som...,"{'drug': [drive, miracle, implanon, sex, take,..."
1,Ethinyl estradiol / norethindrone,birthcontrol terrible years old given since di...,"{'drug': [ask, times, partner, insurance, pay,..."
2,Nexplanon,started nexplanon months ago minimal amount co...,"{'drug': [reacts, well, body, get, pill, inser..."
3,Levonorgestrel,mirena put almost year ago insertion breeze ne...,"{'drug': [review, dilated, another, dr, inject..."
4,Ethinyl estradiol / levonorgestrel,pill many years doctor changed rx chateal effe...,"{'drug': [way, reading, body, daily, hair, est..."


In [27]:
display_df(bc_df)

Etonogestrel:
----------------------------------------------------------------------------------------
 
Key:drug, Values:[drive, miracle, implanon, sex, take, days, cycle, normal, chance, got, day, luck, developed, years, reacts, well, body, get, affects, delivery, effects, implanon, daily, weekly, recommend, understand, effects, would, never, went, nexplanon, exact, laughing, wrap, nausea, object, nexplanon, awful, foreign, aligns, sure, loose, wife, like, child, still, interactions, efficacy, low, outweigh, fingers, implant, good, blood, numbness, removal, month, weight, hate, nexplanon, reading, try, reviews, inserted, daughter, yrs, awful, removed, stitches, pill, months, months, dry, skin, irrititable, terrible, keep, removed, works, normalize, expected, happy, understand, associated, gain, blame, caused, convinced, included, depressed, sensitive, hormonal, recommend, got, increased, would, nov]

Key:pill, Values:[put, bleed, complaint, taking, pregnant, matters, taking, worry, t

In [28]:
display_df(anxiety_df)

Escitalopram:
----------------------------------------------------------------------------------------
 
Key:drug, Values:[works, miracle, sleep, things, worked, years, well, postpartum, gained, weighed, better, sure, evaluate, time, mouth, dry, chance, losing, job, effects, far, side, bed, great, great, thought, miracle, personally, made, days, miracle, anxiety, take, patient, vitamin, periods, days, posted, taking, day, tasks, could, withdrawal, increased, allowed, gain, horrible, became, helps, additional, effects, prescribed, mild, years, body, mean, something, abuse, medication, takes, results, time, get, life, stick, amazing, anxiety, dealing, life, brain, suffered, often, relationships, gave, mind, silent, life, back, made, took, make, doomed, tried, wait, saved, etc, please, australia, effect, flatulence, cheap, live, subsidised, describe, amazing, anxiety, days, brought, around, calmer, air, experience, miracle, company, lexapro, anyone, nausea, awhile, stuck, staple, anxiety,


Key:decrease, Values:[need, marked, memory, makes, hr, taking, dosage, helped, readers, medicine, seems, need, medicine, difficult, withdraw]

Key:lessen, Values:[alcohol, drug, anxiety]

Key:reduce, Values:[dose, frequency, go, significantly, depression, effects, days, without]

Key:drop, Values:[anxiety, half, experience]

Key:cut, Values:[klonopin, taper, weaned, said, could, also, half, take, basis, allowed, back, clonazepam, helps, sleep, pills, bed]

Key:reduction, Values:[period, gradual]

Key:increase, Values:[dose, everyone, different, opened, hoping, poorly, dose, seen, major, dosage, dosage, relaxed, need, body, uncontrollably, take, need, years, felt, dose, ptsd, dosage, klonopin, urge, dose, feel, increases, slightly, dosage, take, dosages, build, take, may, dose, work, suffered, keep, works, never, desire, simply, makes, feel, felt, need, ago, told, pill, slowly, days, seeing, drowsy, starting, take]

Key:improve, Values:[helped, life, take, people, life, help, mood]

Ke